In [ ]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='abc',
    password='xyz'
)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, StoppingCriteria, StoppingCriteriaList
from huggingface_hub import login
import torch._dynamo

# -----------------------------
# LOGIN
# -----------------------------
my_token = "hf_xxxxxxx"
login(token=my_token)

# -----------------------------
# CONFIGURATION
# -----------------------------
MODEL_NAME = "defog/sqlcoder-7b-2"

# Load in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print(f"Loading model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=my_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    token=my_token,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto"
)

torch.set_float32_matmul_precision("high")

#conflicting function
#class StopOnSemicolon(StoppingCriteria):
#    def __call__(self, input_ids, scores, **kwargs):
#        decoded = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#        return ";" in decoded

#stopping_criteria = StoppingCriteriaList([StopOnSemicolon()])

Loading model: defog/sqlcoder-7b-2


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [2]:
schema_info = """superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion UInt32,
    superficie_total Float64,
    superficie_forestal Float64,
    superficie_no_forestal Float64,
    superficie_con_arbolado Float64,
    area_cubierta_por_bosque Float64,
    area_cubierta_por_selva Float64,
    area_cubierta_por_manglar Float64,
    superficie_cubierta_por_otras_areas_arboladas Float64,
    area_cubierta_por_matorral_xerofilo Float64,
    area_cubierta_por_otras_areas_forestales Float64,
    superficie_destinada_a_actividades_agricolas_de_humedad Float64,
    superficie_destinada_a_actividades_agricolas_de_riego Float64,
    superficie_destinada_a_actividades_agricolas_de_temporal Float64,
    superficie_de_cuerpos_de_agua Float64,
    superficie_destinada_a_actividades_acuicolas Float64,
    area_de_pastizales_cultivados Float64,
    area_de_pastizales_inducidos Float64,
    superficie_sin_vegetacion_visible Float64,
    superficie_desprovista_de_vegetacion Float64,
    superficie_ocupada_por_asentamientos_humanos Float64
)
"""

In [3]:
entidad_federativa = "['Aguascalientes', 'Baja California Norte', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de Mexico', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas']"

In [4]:
# -----------------------------
# MAIN FUNCTION
# -----------------------------
def nl_to_sql(request: str) -> str:
    prompt = f"""You are a SQL generator for PostgreSQL.
    Given a user request in natural language (in Spanish), respond with exactly one valid SQL query. Return ONLY the SQL query, no explanations, no markdown formatting.
    
    SCHEMA:
    Table: superficie_bd.superficie_forestal
    {schema_info}

    IMPORTANT RULES:

    1. GRANULARITY:
      - The table has one row per 'entidad_federativa' (state)
      - Available states: {entidad_federativa}

    2. TERMINOLOGY:
      - "entidad federativa" = "estado" (state)
      - "México" (without qualifiers) = the entire country (aggregate all states)
      - "Estado de México" = a specific state
      - "Ciudad de México" = a specific state (different from México and Estado de México)

    3. WHEN TO AGGREGATE:
      - If user asks about "México" alone: Use SUM() without GROUP BY
      - If user asks about specific state(s): Use WHERE with exact state name(s)
      - If user asks about "all states" or "each state": Use GROUP BY entidad_federativa

    4. COLUMNS:
      - All columns except 'entidad_federativa' represent surface area in hectares
      - 'superficie_total' = total area of each state
      - If user doesn't specify columns, select the most relevant based on context

    5. FORMATTING:
      - Always use: SELECT ... FROM superficie_bd.superficie_forestal
      - Use exact column and state names from the schema
      - For state filtering: WHERE entidad_federativa = 'exact_name'
      - For multiple states: WHERE entidad_federativa IN ('state1', 'state2')
      - Add ORDER BY when comparing or listing multiple results

    6. RULE EXAMPLES:
      - "superficie de México" → SUM(superficie_total) without WHERE
      - "superficie de Estado de México" → WHERE entidad_federativa = 'Estado de México'
      - "superficie forestal por estado" → SELECT entidad_federativa, superficie_forestal... GROUP BY entidad_federativa


    REQUEST EXAMPLE:
    User request: Dame la superficie de otras areas arboladas en la entidad federativa Sonora.
    SQL: SELECT
      superficie_cubierta_por_otras_areas_arboladas
    FROM
      superficie_bd.superficie_forestal
    WHERE
      entidad_federativa = 'Sonora';

    REQUEST EXAMPLE:
    User request: ¿Cuál es la entidad federativa con más superficie cubierta por bosque?
    SQL: SELECT
      entidad_federativa,
      superficie_cubierta_por_bosque
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_bosque
    DESC
    LIMIT 1;

    REQUEST EXAMPLE:
    User request: ¿Qué estado tiene la menor área de agua?
    SELECT
      entidad_federativa,
      superficie_cubierta_por_cuerpos_de_agua
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_cuerpos_de_agua
    ASC
    LIMIT 1;

    REQUEST EXAMPLE:
    User request: ¿Cuál es el total de superficie de matorral xerófilo?
    SQL: SELECT
      SUM(superficie_cubierta_por_matorral_xerofilo)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la razón de superficie de manglar entre superficie total por entidad?
    SQL: SELECT
      entidad_federativa,
      superficie_cubierta_por_manglar/superficie_total
    FROM 
      superficie_bd.superficie_forestal
    ORDER BY
      superficie_cubierta_por_manglar/superficie_total
    DESC;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la superficie cubierta por bosque en el Estado de México?
    SQL: SELECT
      superficie_cubierta_por_bosque
    FROM
      superficie_bd.superficie_forestal
    WHERE
      entidad_federativa = 'Estado de Mexico';

    REQUEST EXAMPLE:
    User request: ¿Cuánto ocupa la superficie destinada a asentamientos humanos?
    SQL: SELECT
      SUM(superficie_ocupada_por_asentamientos_humanos)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la superficie forestal en la república?
    SQL: SELECT
      SUM(superficie_forestal)
    FROM
      superficie_bd.superficie_forestal;

    REQUEST EXAMPLE:
    User request: ¿Cuál es la razón de superficie cubierta por otras áreas arboladas entre superficie total en el país?
    SQL: SELECT
      SUM(superficie_cubierta_por_otras_areas_arboladas)/SUM(superficie_total)
    FROM 
      superficie_bd.superficie_forestal;

    Generate only the SQL query.

    User request: {request}

    SQL:
    """

    # 👇 Match input device to model’s first parameter device
    model_device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(model_device)

    gen_kwargs = {
        "max_new_tokens": 128,
        "do_sample": False,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        #"stopping_criteria": stopping_criteria #conflicting line
    }

    @torch._dynamo.disable
    def safe_generate():
        with torch.no_grad():
            return model.generate(**inputs, **gen_kwargs)

    output_ids = safe_generate()
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    sql = full_output.replace(prompt, "").strip()
    if ";" in sql:
      sql = sql.split(";")[0] + ";"
    
    return sql

In [19]:

query = nl_to_sql("Dame la superficie cubierta por agua en Jalisco")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_cubierta_por_cuerpos_de_agua FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Jalisco';


In [20]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_cubierta_por_cuerpos_de_agua',)


[(161704.7867,)]

In [21]:
query = nl_to_sql("¿Cuál es la superficie cubierta por bosque en el Estado de México?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_cubierta_por_bosque FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Estado de Mexico';


In [22]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_cubierta_por_bosque',)


[(623624.7159,)]

In [23]:
query = nl_to_sql("Muestra la razón de cuerpos de agua entre superficie total para Colima")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_cubierta_por_cuerpos_de_agua)/SUM(superficie_total) AS ratio FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Colima';


In [24]:
result = client.query(query)

print(result.column_names)
result.result_rows

('ratio',)


[(0.016827571244275315,)]

In [25]:
query = nl_to_sql("¿Cuál es la razón del total de superficie cubierta por otras áreas forestales entre la población total?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_cubierta_por_otras_areas_forestales)/NULLIF(SUM(poblacion), 0) AS ratio FROM superficie_bd.superficie_forestal;


In [26]:
result = client.query(query)

print(result.column_names)
result.result_rows

('ratio',)


[(0.12091184392137898,)]

In [27]:
query = nl_to_sql("¿Cuál es el estado con la mayor superficie destinada a actividades agrícolas de humedad?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_destinada_a_actividades_agricolas_de_humedad FROM superficie_bd.superficie_forestal ORDER BY superficie_destinada_a_actividades_agricolas_de_humedad DESC LIMIT 1;


In [28]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_destinada_a_actividades_agricolas_de_humedad')


[('Veracruz', 74343.30061)]

In [29]:
query = nl_to_sql("Área ocupada por asentamientos humanos")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(superficie_ocupada_por_asentamientos_humanos) AS superficie_ocupada_por_asentamientos_humanos FROM superficie_bd.superficie_forestal;


In [30]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_ocupada_por_asentamientos_humanos',)


[(2410991.0339599997,)]

In [31]:
query = nl_to_sql("¿Cuáles son las 3 entidades con mayor superficie sin vegetación visible?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT
      entidad_federativa, superficie_sin_vegetacion_visible FROM superficie_bd.superficie_forestal ORDER BY superficie_sin_vegetacion_visible DESC LIMIT 3;


In [32]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_sin_vegetacion_visible')


[('Baja California Norte', 304021.6308),
 ('Sonora', 136439.0437),
 ('Chihuahua', 93518.89663)]

In [33]:
query = nl_to_sql("¿Qué entidad federativa tiene la menor área de bosque?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_cubierta_por_bosque FROM superficie_bd.superficie_forestal ORDER BY superficie_cubierta_por_bosque ASC LIMIT 1;


In [34]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_cubierta_por_bosque')


[('Yucatan', 0.0)]

In [35]:
query = nl_to_sql("¿Cuál es la población total en México?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT SUM(poblacion) AS total_population FROM superficie_bd.superficie_forestal;


In [36]:
result = client.query(query)

print(result.column_names)
result.result_rows

('total_population',)


[(131014024,)]

In [39]:
query = nl_to_sql("¿Cuál es la entidad federativa con menor área de arbolado?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT entidad_federativa, superficie_con_arbolado FROM superficie_bd.superficie_forestal ORDER BY superficie_con_arbolado ASC LIMIT 1;


In [40]:
result = client.query(query)

print(result.column_names)
result.result_rows

('entidad_federativa', 'superficie_con_arbolado')


[('Ciudad de Mexico', 42691.40171)]

In [43]:
query = nl_to_sql("¿Cuál es el área de Oaxaca?")
print("Generated SQL:\n", query)

Generated SQL:
 SELECT superficie_total FROM superficie_bd.superficie_forestal WHERE entidad_federativa = 'Oaxaca';


In [44]:
result = client.query(query)

print(result.column_names)
result.result_rows

('superficie_total',)


[(9376581.998,)]